In [ ]:
!pip install -q tensorflow-gpu==2.0.0-rc1

In [ ]:
!pip install transformers

In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
base_path = '/content/gdrive/My Drive/Colab Notebooks'

In [6]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
train_input_data = 'nsmc_train_input.npy'
train_label_data = 'nsmc_train_label.npy'
data_configs = 'nsmc_data_configs.json'

input_data = np.load(open(data_in_path + train_input_data, 'rb'))
label_data = np.load(open(data_in_path + train_label_data, 'rb'))
prepro_configs = json.load(open(data_in_path + data_configs, 'r'))

In [7]:
test_split = 0.1
seed = 100
batch_size = 50
num_epochs = 16

input_train, input_eval, label_train, label_eval = train_test_split(input_data,
                                                                    label_data,
                                                                    test_size=test_split,
                                                                    random_state=seed)

In [23]:
model_name = 'convnet'
max_length = input_data.shape[1]

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
dataset = dataset.shuffle(buffer_size=len(input_train))
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(len(input_train))

validation_dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
validation_dataset = validation_dataset.shuffle(buffer_size=len(input_eval))
validation_dataset = validation_dataset.batch(batch_size)
dataset = dataset.shuffle(len(input_eval))

In [ ]:
model_name = 'bert-base-multilingual-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name) # 모델 학습 불러오기

In [13]:
model = Model(**kargs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model.build(input_shape=(batch_size, max_length))
model.summary()

Model: "conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  5600896   
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  38500     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  51300     
_________________________________________________________________
conv1d_5 (Conv1D)            multiple                  64100     
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  75250  

In [49]:
test_input_data = 'nsmc_test_input.npy'
test_label_data = 'nsmc_test_label.npy'

test_input_data = np.load(open(data_in_path + test_input_data, 'rb'))
test_label_data = np.load(open(data_in_path + test_label_data, 'rb'))

In [51]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
test_dataset = test_dataset.batch(BATCH_SIZE)

In [53]:
test_output = model.predict(test_dataset)

In [54]:
test_output = np.array(test_output)

In [55]:
test_output

array([[0.9836465 ],
       [0.54142475],
       [0.26841763],
       ...,
       [0.7737855 ],
       [0.00120722],
       [0.0788133 ]], dtype=float32)